# 第九章 魔法方法、特性和迭代器

## 构造函数
* 对象将在创建后自动调用他们
* 命名方法：\__init__

## 析构函数
* 在对象被销毁（垃圾收集）前调用
* 命名方法\__del__

## 继承超类的构造函数
使用函数super（）

In [3]:
class Bird:
    def __init__(self):
        self.hungry = True
    def eat(self):
        if self.hungry:
            print('Aaaaaaaaaaaaah')
        else:
            print('No Thanks')

class SongBird(Bird):
    def __init__(self):
        super().__init__()
        self.sound = 'Squawk!'
    def sing(self):
        print(self.sound)


In [4]:
sb = SongBird()
sb.sing()

Squawk!


In [5]:
sb.eat()

Aaaaaaaaaaaaah


## 元素访问

### 基本的序列和映射协议
序列和映射基本上时元素（item）的集合，要实现他们的行为（协议），不可变对象需要实现两个方法，而可变对象需要实现4个
* \__len__(self):返回集合包含的项数
* \__getitem__(self, key) : 返回指定键相关联的值
* \__setitem__(self, key, value ):以键值对的方式储存值，仅当对象可变的时候才需要实现
* \__delitem__(self, key): 删除与key相关联的值，仅当对象可变的时候才需要实现

* 对于序列，如果键为负数，应该从末尾往前数， x\[-n] = x\[len(x) - n]
* 如果键的类型不合适，可能引发TypeError异常
* 索引不在正确的范围内的话应引发IndexError异常

In [24]:
class ArithmeticSequence:
    '创建一个无穷序列'
    def __init__(self, start = 0, step = 1):
        '初始化序列，开端，步长和改变的值'
        self.start = start
        self.step = step
        self.changed = {}

    def check_index(self, key):
        '合法性检查，如果输入的值不是int类型且大于0的话就会报错'
        if not isinstance(key, int): raise TypeError
        if key < 0: raise IndexError

    def __getitem__(self, key):
        '返回对应的值'
        self.check_index(key)   # 合法性检查
        try:
            return self.changed[key]  # 如果修改过的话就返回修改后的值，没有修改过的话则报错，引发KeyError异常
        except KeyError:                # 没有修改过的话按照等差数列计算结果
            return self.start + key * self.step

    def __setitem__(self, key, value): 
        '修改对应的数值'
        self.check_index(key)
        self.changed[key] = value


In [25]:
s = ArithmeticSequence(1, 2)
s[4]

9

In [26]:
s[4] = 2

In [27]:
s[4]

2

In [28]:
s[5]

11

### 从list，dict和str派生
如果只想定制某种操作行为，就没必要重新实现所有方法，只需要从现有的类中继承

In [59]:
class CounterList(list):
    def __init__(self, *args):
        super().__init__(*args)
        self.counter = 0
    def __getitem__(self, index):
        self.counter += 1
        return super().__getitem__(index)

In [60]:
cl = CounterList(range(10))
cl

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [61]:
cl.counter

0

In [62]:
cl[4]

4

In [63]:
cl.counter

1

In [64]:
cl[4] + cl[9]

13

In [65]:
cl.counter

3

## 特性

通过存取方法定义的属性称为**特性**   
函数property将存取方法作为参数，获取方法在前设置方法在后

In [67]:
class Rectangle:
    def __init__(self):
        self.width = 0
        self.height = 0
    def set_size(self, size):
        self.width, self.height = size
    def get_size(self):
        return self.width, self.height
    size = property(get_size, set_size)

In [68]:
r = Rectangle()
r.width = 5
r.height = 10
r.size

(5, 10)

In [69]:
r.size = 150, 100
r.width

150

## 迭代器
方法\__iter\__返回一个迭代器，他是包含方法\__next\__的对象，调用这个方法时刻不提供任何参数，而调用方法\__next__时，返回下一个值

In [70]:
class fibs:
    'fibnacci 数列'
    def __init__(self):
        self.a = 0
        self.b = 1
    
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        return self.a
    
    def __iter__(self):
        return self

In [72]:
fib = fibs()
for f in fib:
    if f > 1000:
        print(f)
        break

1597


通过可迭代对象调用内置函数iter可获得一个迭代器

In [25]:
it = iter([1, 2, 4])
for i in it:
    print(i)
it = iter([1, 2, 4])
next(it)

1
2
4


1

### 从迭代器创建序列

In [77]:
it = iter([1, 2, 4])
list(it)

[1, 2, 4]

## 生成器

### 生成器是一种用普通函数语法生成的迭代器    
包含yield语句的函数都被成为**生成器**，，生成器不是使用return语句返回一个值，而是可以生成多个值，每次一个。每次使用yield生成一个值后，函数都将被冻结，即在此终止执行，等待被重新唤醒。被重新唤醒后，函数将从停止的地方继续执行

In [14]:
def flatten(nested):
    for sublist in nested:
        for element in sublist:
            yield element

In [3]:
help('yield')

The "yield" statement
*********************

   yield_stmt ::= yield_expression

A "yield" statement is semantically equivalent to a yield expression.
The yield statement can be used to omit the parentheses that would
otherwise be required in the equivalent yield expression statement.
For example, the yield statements

   yield <expr>
   yield from <expr>

are equivalent to the yield expression statements

   (yield <expr>)
   (yield from <expr>)

Yield expressions and statements are only used when defining a
*generator* function, and are only used in the body of the generator
function.  Using yield in a function definition is sufficient to cause
that definition to create a generator function instead of a normal
function.

For full details of "yield" semantics, refer to the Yield expressions
section.



In [47]:
nested = [[1,2], [3,4],[5]]
for num in flatten(nested):
    print(num)


1
2
3
4
5


## 生成器推导（也叫生成器表达式）
* 工作原理与列表推导相同，但不是创建一个列表（即不立即执行循环），而是返回一个生成器，是你能够逐步执行计算
* 与列表推导不同的是使用圆括号()而不是方括号[]

In [48]:
g = ((i+2)**2 for i in range(2, 27))

In [49]:
next(g)

16

In [50]:
sum((i+2)**2 for i in range(2, 27))

7700

In [75]:
# 有单元素检查的生成器
def flatten(nested):
    try:
        for lists in nested:                    #拆分出list的子list（输入的肯定是一个list，如果是单元素会报错进入错误处理）
            for element in flatten(lists):      #对list中的所有子list进行迭代，如果子list是单元素flatten（lists）会返回这个单元素,否则返回list的子list遍历后的结果
                yield element
    except TypeError:
        yield nested


In [82]:
a = [[1,2],[3,4],5, 6]
for num in flatten(a):
    print(num)


1
2
3
4
5
6


In [83]:
a = 'Hello'  #字符串可迭代
for b in a:
    print(b)

H
e
l
l
o


### 通用生成器
生成器包含yield的函数，调用时不会执行函数体内的代码，而是返回一个迭代器，每次请求值时，都将执行生成器代码，知道遇到yield或者return。     
yield意味着生成一个值，return意味着生成器立即停止执行

### 生成器方法
外部世界可以访问生成器的方法森达，类似于next，但接受一个参数（要发送的消息，可以是任何对象）     
在挂起的生成器内部，yield可能用作表达式而不是语句。换而言之，当生成器重新运行时，yield返回一个值--通过send从外部世界发送的值。如果使用的时next，yield将返回None    
send会使得生成器继续运行,返回生成器的下一项

In [99]:
def repeater(value):
    while True:
        new = (yield value)
        if new is not None: value = new

In [100]:
r = repeater(42)
next(r)

42

In [101]:
r.send("Hello world")

'Hello world'

In [102]:
next(r)

'Hello world'

### 方法throw用于在生成器（yield表达式）处引发异常，调用时可以提供一个异常类型，一个可选值和一个traceback对象

### 方法close用于停止生成器，调用时无需提供任何参数

## 八皇后问题
要将八个皇后放在一个 8 * 8 的棋盘上，条件是任意两个皇后不可以位于同一行、同一列或者同一斜线上

In [163]:
def conflict(state, NextX):
    '检查是否新加入的NextX与之前已经放好的棋子有冲突，新加入的棋子纵坐标为len(state)，横坐标为NextX，返回值True(有冲突)或者False（无冲突）'
    NextY = len(state)
    for i in range(NextY):  
        if abs(NextX - state[i]) in (0, NextY - i):  
            #如果横坐标差为0（位于同一列上）或者等于横坐标差（位于对角线上）说明有冲突，返回True，否则返回False
            return True
    return False

def queen(num, state = []):         
    'num为正方形棋盘的长度，state列表表示每个棋子摆放的位置，state[x] = y，意思是这个棋子横坐标x，纵坐标为y'
    if len(state) == num -1:
        # 如果开始计算列表的最后一行的话则应该在计算后返回state状态
        for nextx in range(num): #循环遍历
            if not conflict(state, nextx):  #冲突检查
                state.append(nextx) #摆放棋子
                yield state
                # 回溯，将之前已经遍历过的改变复原
                state.pop()
    else:
        # 如果列表没有满，则返回迭代后的结果
        for nextx in range(num):                    #循环遍历
            if not conflict(state, nextx):          #冲突检查
                state.append(nextx)                 #摆放棋子
                for results in queen(num, state):   #返回一个生成器，对返回的生成器再次进行递归遍历
                    yield results
                state.pop()                         #回溯，将之前已经遍历过的改变复原
    

In [164]:
k =0
for i in queen(8):
    k += 1
    print(i)
print(k)

[0, 4, 7, 5, 2, 6, 1, 3]
[0, 5, 7, 2, 6, 3, 1, 4]
[0, 6, 3, 5, 7, 1, 4, 2]
[0, 6, 4, 7, 1, 3, 5, 2]
[1, 3, 5, 7, 2, 0, 6, 4]
[1, 4, 6, 0, 2, 7, 5, 3]
[1, 4, 6, 3, 0, 7, 5, 2]
[1, 5, 0, 6, 3, 7, 2, 4]
[1, 5, 7, 2, 0, 3, 6, 4]
[1, 6, 2, 5, 7, 4, 0, 3]
[1, 6, 4, 7, 0, 3, 5, 2]
[1, 7, 5, 0, 2, 4, 6, 3]
[2, 0, 6, 4, 7, 1, 3, 5]
[2, 4, 1, 7, 0, 6, 3, 5]
[2, 4, 1, 7, 5, 3, 6, 0]
[2, 4, 6, 0, 3, 1, 7, 5]
[2, 4, 7, 3, 0, 6, 1, 5]
[2, 5, 1, 4, 7, 0, 6, 3]
[2, 5, 1, 6, 0, 3, 7, 4]
[2, 5, 1, 6, 4, 0, 7, 3]
[2, 5, 3, 0, 7, 4, 6, 1]
[2, 5, 3, 1, 7, 4, 6, 0]
[2, 5, 7, 0, 3, 6, 4, 1]
[2, 5, 7, 0, 4, 6, 1, 3]
[2, 5, 7, 1, 3, 0, 6, 4]
[2, 6, 1, 7, 4, 0, 3, 5]
[2, 6, 1, 7, 5, 3, 0, 4]
[2, 7, 3, 6, 0, 5, 1, 4]
[3, 0, 4, 7, 1, 6, 2, 5]
[3, 0, 4, 7, 5, 2, 6, 1]
[3, 1, 4, 7, 5, 0, 2, 6]
[3, 1, 6, 2, 5, 7, 0, 4]
[3, 1, 6, 2, 5, 7, 4, 0]
[3, 1, 6, 4, 0, 7, 5, 2]
[3, 1, 7, 4, 6, 0, 2, 5]
[3, 1, 7, 5, 0, 2, 4, 6]
[3, 5, 0, 4, 1, 7, 2, 6]
[3, 5, 7, 1, 6, 0, 2, 4]
[3, 5, 7, 2, 0, 6, 4, 1]
[3, 6, 0, 7, 4, 1, 5, 2]


## 函数
iter(obj)                            从可迭代对象创建一个迭代器     
next(it)                            让迭代器前进并返回下一个元素     
property(fget, fset, fdel, doc)     返回一个特性，所有的元素都是可选的     
super(class,obj)                    返回一个超类的关联实例

# 本章完